In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
os.chdir('/content/drive/My Drive')

### **GROUND TRUTH VALUES**

*Importing and reading the ground-truth dataset which is in .csv format*

In [ ]:
Ground_Truth_train = pd.read_csv('/content/drive/My Drive/CA684_Assignment/Dev-set/ground-truth.csv')
Ground_Truth_train

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10
...,...,...,...,...,...
5995,video7488.webm,0.921,33,1.000,9
5996,video7489.webm,0.909,53,0.839,31
5997,video7491.webm,0.713,33,0.818,11
5998,video7492.webm,0.954,34,1.000,16


*Checking the correlation between the columns of ground-truth.csv*

In [ ]:
Ground_Truth_train.corr()

,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
short-term_memorability,1.000000,-0.021178,0.325002,-0.013858
nb_short-term_annotations,-0.021178,1.000000,-0.047093,0.285548
long-term_memorability,0.325002,-0.047093,1.000000,0.036051
nb_long-term_annotations,-0.013858,0.285548,0.036051,1.000000


*As seen in the above output annotations doesn't have high correlation so we are drooping them*

In [ ]:
Ground_Truth_train = Ground_Truth_train.drop(['nb_short-term_annotations', 'nb_long-term_annotations'],axis=1)

In [ ]:
Ground_Truth_train

,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900
...,...,...,...
5995,video7488.webm,0.921,1.000
5996,video7489.webm,0.909,0.839
5997,video7491.webm,0.713,0.818
5998,video7492.webm,0.954,1.000


### **C3D_DEVSET**

*Defining a method to read the C3D features from the given .txt files*

In [ ]:
def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D

*Calling a method to get the C3D features and creating the array lists using loop to store them.*

In [ ]:
my_numlist= [] # Creating a list to store the C3D features
my_namelist = [] #Creating a list to store the videos names
path = 'Dev-set/C3D/*.txt'
for filename in glob.glob('Dev-set/C3D/*.txt'):
    s= read_C3D(filename)
    my_numlist.append(s)
    my_namelist.append(((filename.split('/')[-1]).split('.')[0])+ '.webm') 
#adding .webm so that it can be similar to the name of the vedioes present in the ground-truth.csv

In [ ]:
len(my_numlist[0]) #Checking the no.of features captured

101

In [ ]:
len(my_numlist) 

6000

*Creating a dataframe to store the extracted C3D features along with corresponding video names*

In [ ]:
c3d_devset = pd.DataFrame(np.array(my_numlist).reshape(6000,101))
c3d_devset["video"] = my_namelist
#creating new column as video and assigning the names of the vidoes

In [ ]:
c3d_devset.head() #includes video names and captions, which will help further while merging these features with ground truth

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,video
0,0.010858,0.010386,0.000000,0.000000e+00,0.000000e+00,0.000000,2.700000e-07,0.000000,1.000000e-08,3.400000e-07,8.000000e-08,1.000000e-08,0.000004,0.000105,0.000000e+00,4.000000e-08,1.000000e-08,0.000000,0.000000,1.331800e-04,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,7.600000e-07,0.000000,9.800000e-07,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,6.000000e-08,0.000021,5.000000e-08,0.000000e+00,1.000000e-08,3.000000e-08,2.000000e-08,0.000000e+00,...,0.970125,0.000016,0.001298,0.000032,0.000001,0.000000,0.000000,1.000000e-08,2.500000e-07,0.000000e+00,0.000000,5.000000e-08,0.000000,1.000000e-08,1.000000e-08,4.200000e-07,0.000000e+00,3.000000e-08,2.000000e-08,0.000000,0.000000,0.000000,6.000000e-08,0.000000,0.000000,9.000000e-08,0.000000,0.000000e+00,1.100000e-07,1.400000e-07,0.000000,0.000000,1.700000e-07,0.000000,0.000000e+00,1.000000e-08,1.300000e-06,2.600000e-06,8.000000e-08,video6632.webm
1,0.000200,0.000065,0.993807,2.000000e-07,4.700000e-07,0.000073,3.700000e-06,0.000337,6.710000e-06,2.290000e-06,6.380000e-06,7.340000e-06,0.000019,0.000007,3.240000e-06,3.810000e-06,1.411000e-05,0.000001,0.000003,2.300000e-07,1.900000e-07,0.002938,5.200000e-07,9.200000e-07,1.136000e-05,0.000011,3.310400e-04,9.200000e-07,0.000022,8.000000e-08,0.000028,0.000013,5.778000e-05,0.000037,1.683000e-05,6.860000e-06,3.990000e-06,8.030000e-06,1.450000e-06,3.800000e-07,...,0.000027,0.000013,0.000077,0.000050,0.000138,0.000012,0.000007,4.100000e-07,5.440000e-06,3.400000e-07,0.000007,8.399000e-05,0.000004,3.970000e-06,5.900000e-07,1.285300e-04,8.800000e-07,2.228000e-05,1.105000e-05,0.000003,0.000015,0.000014,4.090000e-06,0.000003,0.000005,4.920000e-06,0.000005,6.400000e-07,2.372000e-05,6.610000e-06,0.000010,0.000002,5.800000e-06,0.000001,1.490000e-06,1.170000e-05,1.500000e-07,8.300000e-07,1.060000e-04,video6634.webm
2,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000008,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.999985,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.120000e-06,video6633.webm
3,0.005959,0.004765,0.003757,5.787100e-04,7.344000e-05,0.000129,4.665300e-04,0.010266,3.028100e-04,2.196500e-04,2.521800e-04,2.963730e-03,0.000032,0.000592,3.260000e-05,1.466430e-03,2.430700e-04,0.000181,0.004019,2.390960e-03,1.206900e-04,0.008488,5.216200e-04,4.164600e-04,1.643420e-02,0.049686,1.470550e-03,5.432000e-05,0.001055,5.442770e-03,0.000173,0.024688,9.640700e-04,0.000880,1.140529e-02,4.522160e-03,8.594500e-04,6.573000e-04,6.942400e-04,1.823110e-03,...,0.000639,0.014109,0.000472,0.002461,0.004046,0.000905,0.000997,2.514660e-03,1.587440e-03,1.125030e-03,0.001008,2.133877e-02,0.000647,1.373010e-03,9.568000e-05,5.635083e-02,1.628000e-05,5.379910e-03,6.020020e-03,0.046499,0.023942,0.020658,3.349520e-03,0.002637,0.000094,1.170705e-02,0.000229,9.789210e-03,2.764390e-03,8.052350e-03,0.003992,0.004137,3.670390e-02,0.000767,3.692100e-04,1.251980e-02,8.422000e-05,1.159825e-02,1.155040e-03,video6645.webm
4,0.005782,0.000306,0.004011,1.007000e-05,1.034000e-05,0.000002,3.160000e-06,0.000003,1.984000e-0

*Merging cleaned ground-truth.csv and extracted C3D features dataframe considering video attribute as a primary key*

In [ ]:
c3d_devset_df = pd.merge(Ground_Truth_train, c3d_devset, on='video')
c3d_devset_df.head()

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,video3.webm,0.924,0.846,0.020249,0.001578,0.000826,0.000945,0.000063,0.000003,0.001162,0.000097,0.000022,0.000010,0.000337,0.000636,1.111700e-04,0.001008,0.000004,0.000631,3.905000e-05,0.000041,0.000091,0.000303,1.541000e-05,0.000032,5.221000e-05,0.000062,0.017465,0.000666,0.000065,0.000054,0.000273,0.001386,0.000003,0.001356,0.000517,2.426120e-03,0.002719,2.770000e-06,0.001557,...,0.096647,0.004458,0.011840,0.000287,0.002195,0.002792,0.000018,0.003365,0.000061,0.000573,0.000273,0.000034,0.006737,0.000021,0.000098,2.502000e-05,0.005768,2.990000e-06,0.000085,0.000607,0.000461,0.001189,0.000270,0.000127,0.000106,0.000154,0.000567,0.000020,0.000894,0.001079,0.001042,0.000161,0.000257,0.046617,0.000156,0.000006,0.000537,0.000339,0.008437,0.000470
1,video4.webm,0.923,0.667,0.000118,0.000891,0.000188,0.000045,0.000063,0.000002,0.000641,0.000011,0.000017,0.000014,0.000336,0.004081,5.900000e-07,0.000411,0.000003,0.000243,6.600000e-07,0.000002,0.000535,0.000278,4.804000e-05,0.001153,2.792000e-05,0.000040,0.022276,0.004529,0.000020,0.000029,0.000045,0.060436,0.000039,0.027208,0.000074,2.719000e-05,0.000623,2.609600e-04,0.000183,...,0.000372,0.000154,0.000148,0.000086,0.000267,0.000014,0.001430,0.004417,0.000064,0.000105,0.003129,0.000055,0.000061,0.000025,0.004283,1.032000e-05,0.000062,2.903000e-05,0.000374,0.000139,0.741702,0.007178,0.000346,0.000104,0.000181,0.000052,0.004839,0.000078,0.042325,0.000623,0.000582,0.000393,0.000864,0.000947,0.000136,0.000007,0.000360,0.000159,0.001025,0.000020
2,video6.webm,0.863,0.700,0.011765,0.000746,0.000784,0.000013,0.000007,0.000028,0.000041,0.000014,0.000041,0.000083,0.000048,0.000080,1.066610e-03,0.000958,0.000011,0.000070,2.992000e-05,0.000038,0.000031,0.000008,9.700000e-07,0.002905,1.849000e-05,0.000013,0.000192,0.000050,0.000532,0.000027,0.000051,0.000010,0.000005,0.000029,0.000033,3.260172e-02,0.000034,7.100000e-07,0.000006,...,0.876291,0.005509,0.008852,0.000399,0.000069,0.013166,0.000015,0.000044,0.000520,0.000046,0.000007,0.000024,0.010491,0.000475,0.000023,5.590000e-06,0.005554,2.500000e-07,0.000057,0.000328,0.000149,0.001844,0.000008,0.000103,0.000030,0.000005,0.000179,0.000001,0.000006,0.000215,0.000224,0.000003,0.000031,0.002538,0.000104,0.000005,0.000064,0.005380,0.001027,0.001384
3,video8.webm,0.922,0.818,0.000223,0.000165,0.000007,0.000016,0.000005,0.000014,0.000154,0.000008,0.000004,0.000009,0.001057,0.000126,3.404000e-05,0.000004,0.000006,0.000012,1.940000e-06,0.000003,0.001759,0.000005,8.330000e-06,0.000027,3.200000e-07,0.000010,0.000137,0.000006,0.000005,0.000008,0.000008,0.000611,0.000166,0.583214,0.000004,2.100000e-07,0.000002,2.136000e-05,0.000003,...,0.000009,0.000007,0.000005,0.000539,0.000035,0.000391,0.000021,0.000043,0.000010,0.000028,0.012471,0.006671,0.001187,0.000002,0.119488,5.600000e-07,0.000004,4.250000e-06,0.000040,0.000148,0.103989,0.058989,0.000002,0.000069,0.000004,0.000130,0.001182,0.000003,0.000006,0.000044,0.000046,0.000009,0.000023,0.000053,0.000048,0.000019,0.000001,0.000004,0.000380,0.000029
4,video10.webm,0.950,0.900,0.000090,0.000615,0.003436,0.001281,0.003551,0.000313,0.000042,0.000018,0.000157,0.000117,0.000256,0.000116,3.234800e-04,0.009888,0.000001,0.000576,3.004000e-05,0.000008,0.002029,0.000016,7.690000e-06,0.007324,2.863000e-05,0.000015,0.035220,0.000153,0.001053,0.000031,0.000026,0.001376,0.000215,0.001759,0.000013,1.698540e-03,0.000248,1.754577e-02,0.000260,...,0.000236,0.000046,0.000027,0.000203,0.010644,0.000252,0.000349,0.001776,0.000006,0.001443,0.000146,0.019003,0.000584,0.000027,0.000692,5.650000e-06,0.000403,2.894700e-04,0.000197,0.001254,0.005257,0.045808,0.000008,0.000296,0.003893,0.000318,0.000048,0.000018,0.000264,0.000026,0.000037,0.000690,0.000171,0.00023

*Locking the extracted C3D features on the dataframe using iloc function*

In [ ]:
c3d_devset_features = c3d_devset_df.iloc[:,3:104].values #
c3d_devset_features.shape #checking the size of the locked area in the dataframe

(6000, 101)

*C3D features from the training set are now ready and they can be used directly into model* 

In [ ]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

*Assigning the values to X and Y variables *

In [ ]:
X = c3d_devset_features
y = Ground_Truth_train.iloc[:, 1:3].values

*Splitting the data into train set and test set*

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

*Fitting the C3D features into Decision Tree Model*

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.087
The Spearman's correlation coefficient is: 0.021


*Fitting the C3D features into Linear Regression Model*

In [ ]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.305
The Spearman's correlation coefficient is: 0.150


*Fitting the C3D features into KNN Model*

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knnRegressor = KNeighborsRegressor(n_neighbors = 100)
knnRegressor.fit(X_train, y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.303
The Spearman's correlation coefficient is: 0.133


*Fitting the C3D features into Random Forest Model*

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators=500)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.338
The Spearman's correlation coefficient is: 0.159


*Random Forest is giving the best values when it's estimators are 500.*

Storing the Predicted and Actual scores of long and short term 

In [ ]:
#Creatign a dataframe to store the predicted values of short and long term
pred = pd.DataFrame(np.array(y_pred).reshape(1200,2), columns = ("Predicted_Short_term","Predicted_Long_term"))

In [ ]:
#Creatign a dataframe to store the actual test set values of short and long term
actual = pd.DataFrame(np.array(y_test).reshape(1200,2), columns = ("Actual_Short_term","Actual_Long_term"))

### **HMP_DEVSET**

*Defining a method to read the HMP features from the given .txt files*

In [ ]:
def read_HMP(fname):
    """Scan HMP(Histogram of Motion Patterns) features from file"""
    with open(fname) as f:
        for line in f:
            pairs=line.split()
            HMP_temp = { int(p.split(':')[0]) : float(p.split(':')[1]) for p in pairs}
    # there are 6075 bins, fill zeros
    HMP = np.zeros(6075)
    for idx in HMP_temp.keys():
        HMP[idx-1] = HMP_temp[idx]            
    return HMP

*Calling a method to get the HMP features and creating the array lists using loop to store them.*

In [ ]:
my_numlist= [] # Creating a list to store the C3D features
my_namelist = [] #Creating a list to store the videos names
path = 'Dev-set/HMP/*.txt'
for filename in glob.glob('Dev-set/HMP/*.txt'):
    s= read_HMP(filename)
    my_numlist.append(s)
    my_namelist.append(((filename.split('/')[-1]).split('.')[0])+ '.webm') 
#adding .webm so that it can be similar to the name of the vedioes present in the ground-truth.csv

In [ ]:
len(my_numlist[0]) 

6075

In [ ]:
len(my_numlist)

6000

*Creating a dataframe to store the extracted HMP features along with corresponding video names*

In [ ]:
hmp_devset = pd.DataFrame(np.array(my_numlist).reshape(6000,6075))
hmp_devset["video"] = my_namelist

*Merging cleaned ground-truth.csv and extracted HMP features dataframe considering video attribute as a primary key*

In [ ]:
hmp_devset_df = pd.merge(Ground_Truth_train, hmp_devset, on='video')

In [ ]:
hmp_devset_df.head()

,video,short-term_memorability,long-term_memorability,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,...,6035,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074
0,video3.webm,0.924,0.846,0.125563,0.024036,0.000314,0.0,0.015864,0.000358,0.0,0.000000,0.000086,0.0,0.0,0.0,0.0,0.002795,0.000054,0.0,0.000000,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000002,0.000173,0.000459,0.000000,0.000148,0.000104,0.000000,0.000121,0.000551,0.000000,0.000114,0.000884,0.000002,0.000116,0.000077,0.000002,0.000027,0.000136,0.000000,0.000000,0.000002,0.000000,0.000091,0.000035,0.000000,0.000163,0.000467,0.000002,0.000010,0.000017,0.000000,0.000393,0.000279,0.000000,0.000289,0.001926,0.000000,0.000086,0.000580,0.000000
1,video4.webm,0.923,0.667,0.007526,0.001421,0.000068,0.0,0.001184,0.000143,0.0,0.000000,0.000079,0.0,0.0,0.0,0.0,0.000246,0.000024,0.0,0.000000,0.000042,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000685,0.000308,0.001054,0.000751,0.000176,0.000062,0.000000,0.000123,0.000398,0.000086,0.000246,0.000433,0.000446,0.000143,0.000053,0.000000,0.000053,0.000099,0.000009,0.000004,0.000033,0.000004,0.000051,0.000035,0.000000,0.000062,0.000358,0.000035,0.000024,0.000083,0.000053,0.000244,0.000066,0.000000,0.000081,0.000617,0.000094,0.000220,0.000762,0.001224
2,video6.webm,0.863,0.700,0.109584,0.018978,0.000289,0.0,0.008774,0.000208,0.0,0.000002,0.000088,0.0,0.0,0.0,0.0,0.002046,0.000061,0.0,0.000000,0.000038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000083,0.000057,0.000158,0.000073,0.000021,0.000009,0.000002,0.000019,0.000095,0.000021,0.000019,0.000090,0.000073,0.000050,0.000024,0.000000,0.000012,0.000021,0.000000,0.000000,0.000002,0.000000,0.000017,0.000007,0.000002,0.000066,0.000203,0.000026,0.000002,0.000040,0.000007,0.000054,0.000045,0.000000,0.000028,0.000291,0.000033,0.000052,0.000258,0.000215
3,video8.webm,0.922,0.818,0.120431,0.013561,0.000277,0.0,0.018974,0.000913,0.0,0.000024,0.000713,0.0,0.0,0.0,0.0,0.002496,0.000149,0.0,0.000011,0.000157,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000210,0.000434,0.000543,0.000412,0.000412,0.000045,0.000003,0.000144,0.000282,0.000037,0.000197,0.000218,0.000157,0.000237,0.000021,0.000000,0.000040,0.000056,0.000008,0.000005,0.000013,0.000019,0.000168,0.000013,0.000000,0.000133,0.000202,0.000029,0.000029,0.000035,0.000059,0.001110,0.000075,0.000008,0.000333,0.000793,0.000101,0.000588,0.000503,0.000452
4,video10.webm,0.950,0.900,0.005026,0.001356,0.000055,0.0,0.000665,0.000029,0.0,0.000000,0.000024,0.0,0.0,0.0,0.0,0.000147,0.000020,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000160,0.000996,0.001604,0.000103,0.000768,0.000215,0.000009,0.000415,0.000926,0.000020,0.000538,0.001178,0.000050,0.000518,0.000169,0.000007,0.000134,0.000169,0.000007,0.000026,0.000046,0.000007,0.000373,0.000088,0.000000,0.000338,0.000441,0.000029,0.000070,0.000149,0.000009,0.000882,0.000200,0.000009,0.000559,0.001097,0.000018,0.000632,0.001128,0.000064


*Locking the extracted HMP features on the dataframe using iloc function*

In [ ]:
hmp_devset_features = hmp_devset_df.iloc[:,3:104].values

In [ ]:
hmp_devset_features.shape

(6000, 101)

*HMP features from the training set are now ready and they can be used directly into model*

In [ ]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

*Assigning the values to X and Y variables*

In [ ]:
X = hmp_devset_features
y = Ground_Truth_train.iloc[:, 1:3].values

*Splitting the data into train set and test set*

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

*Fitting the HMP features into Decision Tree Model*

In [ ]:
from sklearn.tree import DecisionTreeRegressor
DTregressor = DecisionTreeRegressor()
DTregressor.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
y_pred = DTregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.085
The Spearman's correlation coefficient is: 0.061


*Fitting the HMP features into Linear Regression Model*

In [ ]:
from sklearn.linear_model import LinearRegression
LMregressor = LinearRegression()
LMregressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
y_pred = LMregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.158
The Spearman's correlation coefficient is: 0.096


*Fitting the HMP features into KNN Model*

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knnRegressor = KNeighborsRegressor(n_neighbors = 100)
knnRegressor.fit(X_train, y_train)
y_pred = knnRegressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.140
The Spearman's correlation coefficient is: 0.000


*Fitting the HMP features into Random Forest Model*

In [ ]:
RFregressor = RandomForestRegressor(n_estimators=500)
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=500, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.223
The Spearman's correlation coefficient is: 0.063


### **Final Predictor Model**

By comparing the above models performed on C3D and HMP dev-sets it is found that C3D has got best spearman's scores when fitted into Random Forest model.
So, C3D test set is used in final predictor model and Random Forest Regressor is performed on that to get the short-term and long-term prediction scores  

*Defining a method to read the C3D testset features from the given .txt files*

In [ ]:
def read_C3D(fname):
    """Scan vectors from file"""
    with open(fname) as f:
        for line in f:
            C3D_test =[float(item) for item in line.split()] # convert to float type, using default separator
    return C3D_test

*Calling a method to get the C3D testset features and creating the array lists using loop to store them.*

In [ ]:
 my_numlist= [] # Creating a list to store the C3D features
my_namelist = [] #Creating a list to store the videos names
path = 'Test-set/C3D_test/*.txt'
for filename in glob.glob('Test-set/C3D_test/*.txt'):
    s= read_C3D(filename)
    my_numlist.append(s)
    my_namelist.append(((filename.split('/')[-1]).split('.')[0])+ '.webm') 
#adding .webm so that it can be similar to the name of the vedioes present in the ground-truth.csv

In [ ]:
len(my_numlist[0])

101

In [ ]:
len(my_numlist)

2000

*Creating a dataframe to store the extracted C3D testset features along with corresponding video names*

In [ ]:
c3d_test = pd.DataFrame(np.array(my_numlist).reshape(2000,101))
c3d_test["video"] = my_namelist
#creating new column as video and assigning the names of the vidoes

In [ ]:
c3d_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,video
0,0.001339,0.000271,0.000396,8.307000e-05,3.563000e-05,4.400000e-07,3.804000e-05,9.790000e-06,1.510000e-06,4.700000e-07,5.432000e-05,5.006000e-05,0.000001,0.000017,8.400000e-07,1.308300e-04,4.221000e-05,1.034700e-04,1.215000e-05,3.909700e-04,1.270000e-05,1.214700e-04,2.900000e-06,1.494000e-05,1.316220e-02,9.115802e-01,4.500000e-07,5.990000e-06,1.202600e-04,1.600200e-04,3.000000e-07,1.724010e-03,8.617000e-05,3.620000e-06,1.636000e-04,2.910000e-06,5.937800e-04,3.772300e-04,8.146800e-04,2.045200e-04,...,2.128100e-04,0.000678,0.000003,0.000087,0.000040,1.532000e-05,0.000174,4.847000e-05,0.000070,3.430000e-05,4.300000e-07,0.000343,8.750000e-06,3.281800e-04,4.200000e-07,0.004115,1.450000e-06,1.028000e-05,3.148000e-05,3.042480e-03,3.889000e-05,2.925400e-04,0.000017,3.071000e-05,1.440000e-06,1.033380e-03,2.564000e-05,1.389678e-02,0.000043,2.050600e-03,3.180000e-06,7.582000e-05,3.406500e-04,1.238000e-05,1.120000e-06,4.935200e-04,0.000074,0.001017,0.000176,video8764.webm
1,0.000001,0.000012,0.000025,1.226000e-05,5.808000e-05,5.716800e-04,1.620000e-06,1.510000e-06,3.001000e-05,3.270000e-06,1.400000e-07,1.003420e-03,0.000391,0.000002,4.340000e-06,7.800000e-07,8.400000e-06,8.700000e-07,1.337300e-04,1.070000e-06,9.800000e-07,6.974269e-02,3.310000e-06,5.284500e-04,2.800000e-07,4.495000e-05,1.260000e-05,7.470000e-06,2.300000e-06,1.510000e-06,3.140989e-02,4.365500e-04,2.710000e-06,1.300000e-07,2.000000e-08,4.524000e-05,1.165700e-04,4.900000e-07,1.020000e-06,1.340000e-06,...,6.500000e-07,0.000005,0.000036,0.000022,0.000002,5.124000e-05,0.000002,1.907000e-05,0.000003,4.000000e-07,3.236100e-04,0.869472,1.204500e-04,5.333320e-03,2.000000e-08,0.000001,1.960000e-06,1.290000e-06,1.661000e-05,2.359000e-05,2.599430e-03,9.730000e-06,0.010963,1.771000e-05,1.130000e-05,1.307000e-05,3.300000e-06,1.520000e-06,0.000003,4.600000e-07,1.900000e-07,2.300000e-07,4.500000e-07,1.550000e-06,3.964600e-04,2.035000e-05,0.000001,0.000026,0.000154,video8758.webm
2,0.001573,0.001398,0.071723,2.488000e-05,1.902000e-05,8.270000e-04,6.838070e-03,1.359300e-04,2.904000e-05,2.393100e-04,5.657900e-04,6.719000e-04,0.000265,0.000454,1.338000e-05,3.156000e-04,5.110800e-04,8.040000e-06,9.750000e-06,3.740000e-06,1.034000e-05,1.801030e-03,1.660000e-06,4.661000e-05,4.602800e-04,3.115000e-05,4.249600e-04,1.333400e-04,1.445600e-03,2.483600e-04,1.712200e-04,5.526000e-05,6.298570e-03,2.643500e-04,5.318000e-05,2.580000e-06,8.437000e-05,3.717900e-04,1.093100e-04,1.313000e-05,...,3.979090e-03,0.003753,0.002594,0.003642,0.030717,3.481370e-03,0.000595,1.133000e-05,0.000506,1.954000e-04,5.344000e-05,0.000353,8.660000e-06,1.816000e-04,9.014000e-05,0.000233,1.899890e-03,3.447500e-04,1.822000e-04,2.582200e-04,1.013890e-03,5.393000e-05,0.000069,1.925300e-04,1.410110e-03,8.689300e-04,1.412000e-05,8.568000e-05,0.001698,4.608000e-05,1.467470e-03,1.279700e-04,1.019300e-04,7.200000e-06,3.577000e-05,1.543550e-03,0.000034,0.000837,0.804485,video8762.webm
3,0.000358,0.003973,0.008803,7.740530e-03,4.029200e-04,1.938100e-04,9.094740e-03,1.752904e-02,2.852800e-04,2.582000e-05,3.279300e-04,1.333743e-02,0.000007,0.000169,1.221300e-04,6.851600e-04,8.164600e-04,3.790000e-06,1.249030e-03,2.968900e-04,1.730540e-03,1.745752e-02,4.575800e-04,1.184056e-02,6.208798e-02,9.157400e-04,3.288000e-05,1.933390e-03,1.349800e-03,3.050290e-03,2.015700e-04,1.381579e-01,2.006050e-03,2.664000e-05,1.112671e-02,2.893917e-02,7.646010e-03,1.535432e-02,6.984520e-03,4.241000e-04,...,8.891300e-04,0.001144,0.003669,0.006591,0.000239,4.588470e-03,0.003227,4.836100e-04,0.000935,6.927425e-02,4.976600e-04,0.016600,2.261100e-04,8.663884e-02,1.025600e-04,0.001129,2.641000e-05,4.527080e-03,3.471980e-03,4.550892e-02,6.679720e-03,1.158050e-03,0.001092,1.939896e-02,1.620340e-03,4.252310e-03,3.154230e-03,5.585500e-04

*C3D testset features from the training set are now ready and they can be used directly into model*

In [ ]:

def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

In [ ]:
X_test = c3d_test

*Fitting the C3D testset features into Random Forest Model*

In [ ]:

from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 150)
RFregressor.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=150, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = RFregressor.predict(X_test)

*Printing the predicted C3D testset features short-term and long-term memorability scores*

In [ ]:
print(y_pred)

[[0.85362    0.77127333]
 [0.81740667 0.73180667]
 [0.84244    0.74698667]
 ...
 [0.86530667 0.7579    ]
 [0.88654    0.84679333]
 [0.88033333 0.81538667]]


*Creating a dataframe to predicted scores and assigning them to predictions variable*

In [ ]:
predictions = pd.DataFrame(np.array(y_pred).reshape(2000,2), columns = ("Short_term_memorability","Long_term_memorability"))

In [ ]:
predictions

,Short_term_memorability,Long_term_memorability
0,0.853620,0.771273
1,0.817407,0.731807
2,0.842440,0.746987
3,0.815647,0.711387
4,0.839380,0.778867
...,...,...
1995,0.887227,0.786307
1996,0.882205,0.795567
1997,0.865307,0.757900
1998,0.886540,0.846793


*Saving the predicting scores in a .CSV file for future reference*

In [ ]:
np.savetxt("predictions.csv", predictions, delimiter = ",")